In [4]:
include("standard_hopfield.jl")
using Statistics, LinearAlgebra, Plots
using DelimitedFiles, Random
using ProfileView

### Without simulated annealing

Now we start from a random pattern and we try to minimize the energy: we would like to end up in a stored pattern.

In [5]:
function matrix_factorization_experiment(;
    N = 1000,
    alpha = 0.02,
    nsweeps = 100,
    ntrials = 1,
    annealing = true)

    # Random.seed!(17)
    M = round(Int, N * alpha)
    ξ = SH.generate_patterns(M, N)
    J = SH.store(ξ)

    ξ_new = zeros(Int, N, M)
    J_new = copy(J)
    M_new = 0 
    for m in 1:M
        for _ in 1:ntrials
            σ = SH.init_pattern(N)
            σ_rec = SH.monte_carlo(J_new, σ; nsweeps = nsweeps, earlystop = 0, β = 100, annealing)
            overlaps = (σ_rec' * ξ) ./ N
            if maximum(abs, overlaps) >= 0.95
                # println("success")
                J_new -= σ_rec * σ_rec' ./ N
                J_new[diagind(J_new)] .= 0
                M_new += 1
                ξ_new[:,m] .= σ_rec
                break
            end
            # println("fail: $(overlaps)")
            # p = histogram(overlaps, bins=-1:0.1:1)
        end
    end
    return ξ_new, M_new
end

matrix_factorization_experiment (generic function with 1 method)

In [6]:
ξ_new, M_new = matrix_factorization_experiment()

([1 1 … 1 -1; -1 -1 … -1 1; … ; 1 -1 … -1 -1; -1 -1 … 1 1], 20)

In [7]:
@profview matrix_factorization_experiment()

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1,000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=2, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint

In [8]:
findmax([-1.0 0.0 -0.13333333333333333])

(0.0, CartesianIndex(1, 2))

### With simulated annealing

In [11]:
M = 10
N = 100
ξ = SH.generate_patterns(M, N)
J = SH.store(ξ)
nsweeps = 100

100